In [31]:
#!pip3 install pulp

In [32]:
import pulp
import pandas as pd

In [33]:
df = pd.read_csv("data/datathon_participants_preprocessed.csv")

In [34]:
df_4 = df[(df['preferred_team_size'] == 4) | (df['preferred_team_size'] == 0)]
df_3 = df[df['preferred_team_size'] == 3]
df_2 = df[df['preferred_team_size'] == 2]
df_1 = df[df['preferred_team_size'] == 1]

In [35]:
n = len(df)
n4 = len(df_4)
n3 = len(df_3)
n2 = len(df_2)
n1 = len(df_1)

print(n)
print(n4)
print(n3)
print(n2)
print(n1)

924
801
98
19
6


## CLUSTERITZACIÓ 4

In [36]:
mida_grup = 4
num_grups = n4 // mida_grup

while n4 % mida_grup != 0:
    random_row = df_3.sample(n=1)
    df2 = pd.concat([df_4, random_row], ignore_index=True)
    df1 = df_3.drop(random_row.index)

    n4 += 1
    n3 -= 1

num_grups = n4 // mida_grup


In [37]:
x = pulp.LpVariable.dicts("x", [(i, j) for i in df_4 for j in range(num_grups)], cat='Binary')
x

{('id', 0): x_('id',_0),
 ('id', 1): x_('id',_1),
 ('id', 2): x_('id',_2),
 ('id', 3): x_('id',_3),
 ('id', 4): x_('id',_4),
 ('id', 5): x_('id',_5),
 ('id', 6): x_('id',_6),
 ('id', 7): x_('id',_7),
 ('id', 8): x_('id',_8),
 ('id', 9): x_('id',_9),
 ('id', 10): x_('id',_10),
 ('id', 11): x_('id',_11),
 ('id', 12): x_('id',_12),
 ('id', 13): x_('id',_13),
 ('id', 14): x_('id',_14),
 ('id', 15): x_('id',_15),
 ('id', 16): x_('id',_16),
 ('id', 17): x_('id',_17),
 ('id', 18): x_('id',_18),
 ('id', 19): x_('id',_19),
 ('id', 20): x_('id',_20),
 ('id', 21): x_('id',_21),
 ('id', 22): x_('id',_22),
 ('id', 23): x_('id',_23),
 ('id', 24): x_('id',_24),
 ('id', 25): x_('id',_25),
 ('id', 26): x_('id',_26),
 ('id', 27): x_('id',_27),
 ('id', 28): x_('id',_28),
 ('id', 29): x_('id',_29),
 ('id', 30): x_('id',_30),
 ('id', 31): x_('id',_31),
 ('id', 32): x_('id',_32),
 ('id', 33): x_('id',_33),
 ('id', 34): x_('id',_34),
 ('id', 35): x_('id',_35),
 ('id', 36): x_('id',_36),
 ('id', 37): x_('id',

Problema d'optimització

In [38]:
optim = pulp.LpProblem("Clustering_de_Persones", pulp.LpMinimize)

# Funció objectiu
optim += pulp.lpSum(
    distancies[i][k] * x[i, j] * x[k, j]
    for j in range(num_grups)
    for i in df_4
    for k in df_4 if i < k
)

NameError: name 'distancies' is not defined

## Restriccions

Restricció 1: Cada persona ha d'estar en un únic grup

In [ ]:
for i in df_4:
    optim += pulp.lpSum(x[i, j] for j in range(num_grups)) == 1

Restricció 2: Cada grup ha de tenir exactament 4 persones (o menys si hi ha persones sobrants)


In [ ]:
for j in range(num_grups):
    optim += pulp.lpSum(x[i, j] for i in df_4) <= mida_grup
    optim += pulp.lpSum(x[i, j] for i in df_4) >= mida_grup


In [ ]:
optim.solve()

In [ ]:
from collections import defaultdict

grups = defaultdict(list)
for i in df_4:
    for j in range(num_grups):
        if pulp.value(x[i, j]) == 1:
            grups[j].append(i)

# Imprimir els grups
for j in sorted(grups):
    print(f"Grup {j+1}: {grups[j]}")

In [ ]:
print("Status:", pulp.LpStatus[optim.status])
print("Cost total:", pulp.value(optim.objective))